In [31]:
import random
import pygame
import csv
import numpy as np
from collections import deque
from numpy import loadtxt
from pygame.locals import *
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
pygame.init()

clock = pygame.time.Clock()
fps = 60

screen_width = 1200
screen_height = 600

screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption('ANN')

# define game variables
tile_size = 100
node_size = 150

In [32]:
class qWorld():
    def __init__(self):
        #Q table parameters
        self.numStates = 16
        self.numActions = 2
        self.gridState = -1
        self.qTable = np.zeros([self.numStates, self.numActions])
        self.init_reward_tables()
        self.init_transition_table()
        self.epsilon = 1.0
        self.gamma = .99
        self.epsilon_min = .01
        self.epsilon_decay = .996
        self.reset_state()
        
        
    #Reinforcement Learning Functions
    def init_reward_tables(self):
        self.rTables = []
        self.rTable = np.zeros([self.numActions,self.numStates])
        
        #ob0 state 1
        self.rTable[1, 0] = 10
        self.rTable[1, 6] = -100
        self.rTable[0, 6] = 100
        self.rTables.append(self.rTable)
        self.rTable = np.zeros([self.numActions,self.numStates])
        
        #ob0 state 2
        self.rTable[1, 0] = 10
        self.rTable[1, 5] = -100
        self.rTable[0, 5] = 100
        self.rTables.append(self.rTable)
        self.rTable = np.zeros([self.numActions,self.numStates])
        
        #ob0 state 3
        self.rTable[1, 0] = 10
        self.rTable[1, 4] = -100
        self.rTable[0, 4] = 100
        self.rTables.append(self.rTable)
        self.rTable = np.zeros([self.numActions,self.numStates])
        
        #ob0 state 4
        self.rTable[1, 0] = 10
        self.rTable[1, 3] = -100
        self.rTable[0, 3] = 100
        self.rTables.append(self.rTable)
        self.rTable = np.zeros([self.numActions,self.numStates])
        
        #ob0 state 5
        self.rTable[0, 0] = 10
        self.rTable[1, 2] = -100
        self.rTable[0, 2] = 100
        self.rTables.append(self.rTable)
        self.rTable = np.zeros([self.numActions,self.numStates])
        
        #ob0 state 6
        self.rTable[0, 0] = 10
        self.rTable[1, 1] = -100
        self.rTable[0, 1] = 100
        self.rTables.append(self.rTable)
        self.rTable = np.zeros([self.numActions,self.numStates])
        
        #ob0 state 7
        self.rTable[0, 0] = 10
        self.rTable[1, 0] = -100
        self.rTable[1, 1] = 100
        self.rTables.append(self.rTable)
        self.rTable = np.zeros([self.numActions,self.numStates])
        
        #ob0 state 8
        self.rTable[0, 0] = 10
        self.rTable[1, 0] = 100
        self.rTables.append(self.rTable)
        self.rTable = np.zeros([self.numActions,self.numStates])
        
        #ob1 state 1
        self.rTable[1, 0] = 10
        self.rTable[0, 6] = -100
        self.rTable[1, 7] = 100
        self.rTables.append(self.rTable)
        self.rTable = np.zeros([self.numActions,self.numStates])
        
        #ob1 state 2
        self.rTable[1, 0] = 10
        self.rTable[0, 5] = -100
        self.rTable[1, 6] = 100
        self.rTables.append(self.rTable)
        self.rTable = np.zeros([self.numActions,self.numStates])
        
        #ob1 state 3
        self.rTable[1, 0] = 10
        self.rTable[0, 4] = -100
        self.rTable[1, 5] = 100
        self.rTables.append(self.rTable)
        self.rTable = np.zeros([self.numActions,self.numStates])
        
        #ob1 state 4
        self.rTable[1, 0] = 10
        self.rTable[0, 3] = -100
        self.rTable[1, 4] = 100
        self.rTables.append(self.rTable)
        self.rTable = np.zeros([self.numActions,self.numStates])
        
        #ob1 state 5
        self.rTable[0, 0] = 10
        self.rTable[0, 2] = -100
        self.rTable[1, 3] = 100
        self.rTables.append(self.rTable)
        self.rTable = np.zeros([self.numActions,self.numStates])
        
        #ob1 state 6
        self.rTable[0, 0] = 10
        self.rTable[0, 1] = -100
        self.rTable[1, 2] = 100
        self.rTables.append(self.rTable)
        self.rTable = np.zeros([self.numActions,self.numStates])
        
        #ob1 state 7
        self.rTable[0, 0] = 10
        self.rTable[0, 0] = -100
        self.rTable[1, 1] = 100
        self.rTables.append(self.rTable)
        self.rTable = np.zeros([self.numActions,self.numStates])
        
        #ob1 state 8
        self.rTable[0, 0] = 10
        self.rTable[1, 0] = 100
        self.rTables.append(self.rTable)
        self.rTable = np.zeros([self.numActions,self.numStates])
        
    def init_transition_table(self):
        self.transition_table = np.zeros([2,2], dtype=int)
        
        self.transition_table[0,0] = 1
        self.transition_table[0,1] = 1
        self.transition_table[1,0] = 1
        self.transition_table[1,1] = 0
        
    def reset_state(self):
        self.state = 0
        return self.state
        
    def step(self, action, obType):
        next_state = self.transition_table[self.state, action]
        if obType == 0:    
            reward = self.rTables[self.gridState][self.state, action]
        elif obType == 1:
            reward = self.rTables[self.gridState][self.state, action]
        self.state = next_state
        return next_state, reward
    
    def act(self):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(2,1)[0]
        
        return np.argmax(self.qTable[self.gridState])
    
    def act_data(self):
        #print(self.qTable[self.gridState])
        return np.argmax(self.qTable[self.gridState])
    
    def update_q_table(self, state, action, reward, next_state):
        q_value = self.gamma * np.amax(self.qTable[next_state])
        q_value += reward
        self.qTable[self.gridState, action] = q_value
        
    def update_epsilon(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [33]:

class Player():
    def __init__(self, x, y):
        self.images_right = []
        self.index = 0
        self.anim_index = 0
        self.counter = 0
        for num in range(1, 4):
            img_right = pygame.image.load(f'img/Run{num}.png')
            img_right = pygame.transform.scale(img_right, (100,100))
            self.images_right.append(img_right)
        self.image = self.images_right[self.index]
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y
        self.width = self.image.get_width()
        self.height = self.image.get_height()
        self.vel_y = 0
        self.jumped = False
        self.direction = 1
        self.reset = False
        #Backpropagation Values
        self.capture = False
        self.bp_ai_mode = False
        self.index = 0
        self.training_data = None
        self.writer = None
        self.model = Sequential()
        #Reinforcement Learning Values
        self.rl_ai_training = False
        self.rl_ai_mode = False
    
    #player/game functions
    def jump(self):
        self.vel_y = -23
        self.jumped = True
        
    def update(self, index):
        dx = 0
        dy = 0
        walk_cooldown = 1
        self.index = index
        
        #get keypresses
        key = pygame.key.get_pressed()
        #Capture Mode
        if key[pygame.K_c]:
            self.training_data = open('bp_training_data.csv', mode='w')
            self.writer = csv.writer(self.training_data, delimiter=',', quotechar='"', 
                                     quoting=csv.QUOTE_MINIMAL, lineterminator='\n')
            self.capture = True
        #BP ANN Mode
        if key[pygame.K_p]:
            self.bp_ai_mode = True
            model = keras.models.load_model('BP_model.h5')
        #Train Back Propagation Model
        if key[pygame.K_b]:
            dataset = loadtxt('bp_training_data.csv', delimiter=',')
            # split into input (X) and output (y) variables
            X = dataset[:,0:5]
            y = dataset[:,5]
            self.model.add(Dense(4, input_dim=5, activation='relu'))
            self.model.add(Dense(1, activation='sigmoid'))
            # compile the keras model
            self.model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
            # fit the keras model on the dataset
            self.model.fit(X, y, epochs=30, batch_size=100)
            # evaluate the keras model
            _, accuracy = self.model.evaluate(X, y)
            print('Accuracy: %.2f' % (accuracy*100))
            #predictions for show
            predictions = np.argmax(self.model.predict(X), axis=-1)
            # summarize the first 15 cases
            for i in range(15):
                print('%s => %d (expected %d)' % (X[i].tolist(), predictions[i], y[i]))
            # save model and architecture to single file
            self.model.save("BP_model.h5")
            print("Saved model to successfully")
        #Start Training Reinforcemnt Learning Model
        if key[pygame.K_t]:
            self.rl_ai_training = True
        if key[pygame.K_r]:
            self.rl_ai_mode = True
            
            
        #Actions
        if key[pygame.K_SPACE] and self.jumped == False and self.bp_ai_mode == False and self.rl_ai_training == False and self.rl_ai_mode == False:
            self.jump()
        elif self.bp_ai_mode == True:
            temp_action = self.model.predict([ob_group.sprites()[index].rect.x, ob_group.sprites()[index].rect.y + 200,
                         ob_group.sprites()[index].rect.y, self.rect.x, self.rect.y,])
            action = np.argmax(temp_action, axis=-1)
            if action == 1 and self.jumped == False:
                self.jump()
        
        
        #handle animations
        self.counter += 1
        if self.counter > walk_cooldown:
            self.counter = 0
            self.anim_index += 1
            if self.anim_index >= len(self.images_right):
                self.anim_index = 0
            if  self.direction == 1:
                self.image = self.images_right[self.anim_index]
            if  self.direction == -1:
                self.image = self.images_left[self.anim_index]
        
        
        #add gravity
        self.vel_y += 1
        if self.vel_y > 10:
            self.vel_y = 10
        dy += self.vel_y
        
        
        #check for collision
        for ob in ob_group:
            if pygame.Rect.colliderect(ob.rect, self.rect):
                self.reset = True
                #print(reset)
        for tile in world.tile_list:
            #check for col in x direction
            if tile[1].colliderect(self.rect.x + dx, self.rect.y, self.width, self.height):
                dx = 0
                
            #check for col in y direction
            if tile[1].colliderect(self.rect.x, self.rect.y + dy, self.width, self.height):
                #check if below the ground i.e. jumping
                if self.vel_y < 0:
                    dy = tile[1].bottom - self.rect.top
                    self.vel_y = 0
                #check if above the ground i.e. falling
                elif self.vel_y >= 0:
                    dy = tile[1].top - self.rect.bottom
                    self.vel_y = 0
                    self.jumped = False
        
        
        #update player coords
        self.rect.x += dx
        self.rect.y += dy
        
        if self.rect.bottom > screen_height:
            self.rect.bottom = screen_height
            dy = 0
            
        #Capture training data
        if self.capture == True:
            temp_data = [ob_group.sprites()[index].rect.x, ob_group.sprites()[index].rect.y + 200,
                         ob_group.sprites()[index].rect.y, self.rect.x, self.rect.y,]
            if(self.jumped):
                temp_data.append('1')
            else:
                temp_data.append('0')
            self.writer.writerow(temp_data)
        
        #draw player onto screen
        screen.blit(self.image, self.rect)
        #pygame.draw.rect(screen, (255,255,255), self.rect, 2)


In [34]:
class Obstacle(pygame.sprite.Sprite):
    def __init__(self, type):
        pygame.sprite.Sprite.__init__(self)
        self.type = type
        img = pygame.image.load('img/wallOb.png')
        img = pygame.transform.scale(img, (100,200))
        self.image = img
        self.rect = self.image.get_rect()
        self.rect.x = 1300
        self.width = self.image.get_width()
        self.height = self.image.get_height()
        self.vel_x = 0
        self.activated = False
        if self.type == 0:
            self.rect.y = 100
        elif self.type == 1:
            self.rect.y = 300
    
    def update(self):
        if self.activated == True:
            self.vel_x = -20
        if self.rect.x <= -100:
            self.vel_x = 0
            self.activated = False
            self.rect.x = 1300
        
        #update coords
        self.rect.x += self.vel_x
        
        pygame.draw.rect(screen, (255,255,255), self.rect, 2)

In [35]:
class World():
    def __init__(self, data):
        self.tile_list = []
        self.bgTile_list = []
        
        #load images
        bg_img = pygame.image.load('img/bgTile.png')
        block_img = pygame.image.load('img/block.png')
        
        row_count = 0
        for row in data:
            col_count = 0
            for tile in row:
                if tile == 0:
                    img = pygame.transform.scale(bg_img, (tile_size, tile_size))
                    img_rect = img.get_rect()
                    img_rect.x = col_count  * tile_size
                    img_rect.y = row_count  * tile_size
                    tile = (img, img_rect)
                    self.bgTile_list.append(tile)
                if tile == 1:
                    img = pygame.transform.scale(block_img, (tile_size, tile_size))
                    img_rect = img.get_rect()
                    img_rect.x = col_count  * tile_size
                    img_rect.y = row_count  * tile_size
                    tile = (img, img_rect)
                    self.tile_list.append(tile)
                col_count += 1
            row_count += 1
            
    def draw(self):
        for tile in self.tile_list:
            screen.blit(tile[0], tile[1])
            #pygame.draw.rect(screen, (255,255,255), tile[1], 2)
        for tile in self.bgTile_list:
            screen.blit(tile[0], tile[1])
            #pygame.draw.rect(screen, (255,255,255), tile[1], 2)
    


In [36]:
world_data = [
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
]


player = Player(100, screen_height - 130)

qworld = qWorld()

#Set up obstacles
ob_group = pygame.sprite.Group()
for i in range(3):
    new_ob = Obstacle(0)
    ob_group.add(new_ob)
for i in range(3):
    new_ob = Obstacle(1)
    ob_group.add(new_ob)

world = World(world_data)

active_counter = 80
run = True
index = 0
maxSteps = 5000
episodes = 10 * maxSteps
scores = deque(maxlen=maxSteps)
step = 1
currentOb = ob_group.sprites()[0]
rl_loaded = False

while run:
    #Training Loop
    if player.rl_ai_training:
        for episode in range(episodes):
            state = qworld.reset_state()
            while step < maxSteps:
                clock.tick(fps)
                active_counter -= 1
    
                world.draw()
    
                if active_counter <= 0:
                    index = random.randint(0, len(ob_group.sprites()) - 1)
                    ob_group.sprites()[index].activated = True
                    currentOb = ob_group.sprites()[index]
                    active_counter = 80
                ob_group.update()
                ob_group.draw(screen)
                
                player.update(index)
                if currentOb.type == 0:
                    if currentOb.rect.x < 1050 and currentOb.rect.x > 900:
                        qworld.gridState = 0
                    elif currentOb.rect.x < 900 and currentOb.rect.x > 750:
                        qworld.gridState = 1
                    elif currentOb.rect.x < 750 and currentOb.rect.x > 600:
                        qworld.gridState = 2
                    elif currentOb.rect.x < 600 and currentOb.rect.x > 450:
                        qworld.gridState = 3
                    elif currentOb.rect.x < 450 and currentOb.rect.x > 300:
                        qworld.gridState = 4
                    elif currentOb.rect.x < 300 and currentOb.rect.x > 150:
                        qworld.gridState = 5
                    elif currentOb.rect.x < 150 and currentOb.rect.x > 0:
                        qworld.gridState = 6
                    elif currentOb.rect.x < 0 and currentOb.rect.x > -100:
                        qworld.gridState = 7
                elif currentOb.type == 1:
                    if currentOb.rect.x < 1050 and currentOb.rect.x > 900:
                        qworld.gridState = 8
                    elif currentOb.rect.x < 900 and currentOb.rect.x > 750:
                        qworld.gridState = 9
                    elif currentOb.rect.x < 750 and currentOb.rect.x > 600:
                        qworld.gridState = 10
                    elif currentOb.rect.x < 600 and currentOb.rect.x > 450:
                        qworld.gridState = 11
                    elif currentOb.rect.x < 450 and currentOb.rect.x > 300:
                        qworld.gridState = 12
                    elif currentOb.rect.x < 300 and currentOb.rect.x > 150:
                        qworld.gridState = 13
                    elif currentOb.rect.x < 150 and currentOb.rect.x > 0:
                        qworld.gridState = 14
                    elif currentOb.rect.x < 0 and currentOb.rect.x > -100:
                        qworld.gridState = 15
                if(player.rl_ai_training):
                    action = qworld.act()
                else:
                    action = qwotld.act_data()
                next_state, reward = qworld.step(action, currentOb.type)
                if action == 1 and player.jumped == False:
                    player.jump()
                if(player.rl_ai_training):
                    qworld.update_q_table(state, action, reward, next_state)
                state = next_state
                if(player.rl_ai_training):
                    qworld.update_epsilon()
                step += 1
                
                if player.reset == True:
                    player.rect.x = 100
                    player.rect.y = screen_height - 130
                    
                    #Set up obstacles
                    ob_group.empty()
                    for i in range(3):
                        new_ob = Obstacle(0)
                        ob_group.add(new_ob)
                    for i in range(3):
                        new_ob = Obstacle(1)
                        ob_group.add(new_ob)
                    player.reset = False
                
                
                #print(world.tile_list)
                
                
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        run = False
                        if player.capture == True:
                            player.training_data.close()
                    active_counter = 80
                
                pygame.display.update()
        #np.savetxt('./RL_qTable.csv', qworld.qTable, delimiter=',')
        print(qworld.qTable)
        with open("./RL_qTable.csv","w+") as my_csv:
            csvWriter = csv.writer(my_csv,delimiter=',')
            csvWriter.writerows(qworld.qTable)
        my_csv.close()
        player.rl_ai_training = False
        print("RL Training Complete")
    
    #main Loop
    clock.tick(fps)
    active_counter -= 1
    
    world.draw()
    
    if active_counter <= 0:
        index = random.randint(0, len(ob_group.sprites()) - 1)
        ob_group.sprites()[index].activated = True
        currentOb = ob_group.sprites()[index]
        active_counter = 80
    ob_group.update()
    ob_group.draw(screen)
    
    player.update(index)
    
    if(player.rl_ai_mode):
        if rl_loaded == False:
            CSVData = open("./RL_qTable.csv")
            qworld.qTable = np.loadtxt(CSVData, delimiter=",")
            CSVData.close()
            rl_loaded = True
            print("RL Q Table Loaded\n")
            print(qworld.qTable)
        if currentOb.type == 0:
            if currentOb.rect.x < 1050 and currentOb.rect.x > 900:
                qworld.gridState = 0
            elif currentOb.rect.x < 900 and currentOb.rect.x > 750:
                qworld.gridState = 1
            elif currentOb.rect.x < 750 and currentOb.rect.x > 600:
                qworld.gridState = 2
            elif currentOb.rect.x < 600 and currentOb.rect.x > 450:
                qworld.gridState = 3
            elif currentOb.rect.x < 450 and currentOb.rect.x > 300:
                qworld.gridState = 4
            elif currentOb.rect.x < 300 and currentOb.rect.x > 150:
                qworld.gridState = 5
            elif currentOb.rect.x < 150 and currentOb.rect.x > 0:
                qworld.gridState = 6
            elif currentOb.rect.x < 0 and currentOb.rect.x > -100:
                qworld.gridState = 7
        elif currentOb.type == 1:
            if currentOb.rect.x < 1050 and currentOb.rect.x > 900:
                qworld.gridState = 8
            elif currentOb.rect.x < 900 and currentOb.rect.x > 750:
                qworld.gridState = 9
            elif currentOb.rect.x < 750 and currentOb.rect.x > 600:
                qworld.gridState = 10
            elif currentOb.rect.x < 600 and currentOb.rect.x > 450:
                qworld.gridState = 11
            elif currentOb.rect.x < 450 and currentOb.rect.x > 300:
                qworld.gridState = 12
            elif currentOb.rect.x < 300 and currentOb.rect.x > 150:
                qworld.gridState = 13
            elif currentOb.rect.x < 150 and currentOb.rect.x > 0:
                qworld.gridState = 14
            elif currentOb.rect.x < 0 and currentOb.rect.x > -100:
                qworld.gridState = 15
        action = qworld.act_data()
        next_state, reward = qworld.step(action, currentOb.type)
        if action == 1 and player.jumped == False:
            player.jump()
        state = next_state
    
    if player.reset == True:
        player.rect.x = 100
        player.rect.y = screen_height - 130
        
        #Set up obstacles
        ob_group.empty()
        for i in range(3):
            new_ob = Obstacle(0)
            ob_group.add(new_ob)
        for i in range(3):
            new_ob = Obstacle(1)
            ob_group.add(new_ob)
        player.reset = False
    
    
    #print(world.tile_list)
    
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False
            if player.capture == True:
                player.training_data.close()
        active_counter = 80
    
    pygame.display.update()

pygame.quit()

RL Q Table Loaded

[[890.85656416 101.52910018]
 [897.24362172 177.34767156]
 [898.2711855  881.94799852]
 [898.2711855  516.56625493]
 [888.2711855  109.30967108]
 [888.2711855  746.8315355 ]
 [ 69.13906218 880.85656416]
 [980.85656416 118.11657437]
 [898.2711855  865.62299594]
 [898.2711855  700.67647661]
 [898.2711855    0.        ]
 [898.2711855  110.41380917]
 [880.85656416 846.8315355 ]
 [880.85656416 874.07250444]
 [  0.           0.        ]
 [  0.           0.        ]]
